# Can we identify faulty or suboptimally performing equipment?

In [49]:
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

gd_train=pd.read_csv("Data/Train_Plant1.csv")
gd_test=pd.read_csv("Data/Test_Plant1.csv")


# NB! sõltuvalt Train/test switch

In [50]:
gd_train[48576:]
#24*4*22*3=6336
#53856-6336= 47520

#alates 2020-06-07

#Test andmestik
#gd=pd.concat([gd_train[48576:],gd_test], ignore_index=True)

#Train andmestik
gd=pd.concat([gd_train,gd_test[:1056]], ignore_index=True)

In [51]:
#Add columns
gd['DC_POWER_DIF_AVG_PER_ALL']=gd['DC_POWER']-gd['DC_POWER'].mean()
gd['AC_POWER_DIF_AVG_PER_ALL']=gd['AC_POWER']-gd['AC_POWER'].mean()
gd['DATE_TIME'] = pd.to_datetime(gd['DATE_TIME'], errors='coerce')
gd['DATE']=pd.to_datetime(gd['DATE_TIME'].dt.date)
gd['TIME']=gd['DATE_TIME'].dt.time

In [52]:
#New columns per inverter 
uurimiseks= gd.copy()
groups = dict(list(uurimiseks.groupby("SOURCE_KEY")))
shift_nr=3
shift_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']
gr_nr=0

#inverter based shifting
for group in groups:
    gr_nr+=1
    dates=groups[group]['DATE'].unique()
    for i in range(shift_nr):
        for j in range(len(shift_columns)):
            groups[group][shift_columns[j]+'_PER_INVERTER_MOMENT_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=(i+1), fill_value=0)
            groups[group][shift_columns[j]+'_PER_INVERTER_DAY_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=(i+1)*96, fill_value=0)
    
    #groups[group]['INVERTER']=group - ebaoluline
    groups[group]['DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    ############ per one day comparison, but note sure if correct
    gr_no2=0
    for date in dates:
        gr_no2 +=1
        d=groups[group][groups[group]['DATE']==date]
        d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
        d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])
        if gr_no2==1:
            date_data=pd.DataFrame(columns = d.keys())
            
        date_data =pd.concat([date_data, d], ignore_index=True)
    #########
    
    if gr_nr==1:
        gen_data=pd.DataFrame(columns = date_data.keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data=pd.concat([gen_data,date_data], ignore_index=True)
    
#gen_data=gen_data.fillna(value=0) - ebaoluline
gen_data

<ipython-input-52-e7170d9f046c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
<ipython-input-52-e7170d9f046c>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])


,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235
1,2020-05-15 00:15:00,1BY6WEcLGh8j5v7,25.084589,22.761668,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235
2,2020-05-15 00:30:00,1BY6WEcLGh8j5v7,24.935753,22.592306,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235
3,2020-05-15 00:45:00,1BY6WEcLGh8j5v7,24.846130,22.360852,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,25.184316,0.000000,22.857507,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235
4,2020-05-15 01:00:00,1BY6WEcLGh8j5v7,24.621525,22.165423,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,25.084589,0.000000,22.761668,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,2020-06-09 22:45:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,22.119533,0.000000,20.250877,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000
54908,2020-06-09 23:00:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,22.125964,0.000000,20.276077,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000
54909,2020-06-09 23:15:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,22.194932,0.000000,20.219128,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000
54910,2020-06-09 23:30:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,22.081917,0.000000,20.148322,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000


# Validation by comparison

In [53]:
#gen_data['AC_POWER'][20000:20004]

In [54]:
#gen_data['AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1'][20000:20004]

In [55]:
#a=gen_data[gen_data['SOURCE_KEY']=='1BY6WEcLGh8j5v7']
#b=pd.concat([a[a['DATE_TIME']=='2020-05-17 13:00:00'],a[a['DATE_TIME']=='2020-05-16 13:00:00']], ignore_index=True)
#b['AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1']

In [56]:
#b['AC_POWER']

In [57]:
#groups[group].keys()

# Other fields

In [58]:
#Group by day labels
'''
* label "FAULTY_according_to_DAILY_YIELD" - yield stays the same, while other inverters still increases
* label "INCORRECT_daily_yield" - yield per day decreases
* columns per inverter day average differences (total are done)
'''

uurimiseks= gen_data.copy()
groups = dict(list(uurimiseks.groupby("DATE")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_AVG_DIF_PER_DAY','AC_AVG_DIF_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_AVG_DIF_PER_DAY'] =  groups[group]['DC_POWER'] -np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_AVG_DIF_PER_DAY'] =  groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    gr_nr+=1
    '''
    Tundub et toodaks vale infot
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_DAY_SHIFT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data2=pd.DataFrame(columns =groups[group].keys())
    
    gen_data2=pd.concat([gen_data2,groups[group]], ignore_index=True)
    
gen_data2

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.000000,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235,-2664.412473,-260.810113
1,2020-05-15 00:15:00,1BY6WEcLGh8j5v7,25.084589,22.761668,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.000000,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235,-2664.412473,-260.810113
2,2020-05-15 00:30:00,1BY6WEcLGh8j5v7,24.935753,22.592306,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.000000,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235,-2664.412473,-260.810113
3,2020-05-15 00:45:00,1BY6WEcLGh8j5v7,24.846130,22.360852,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,22.857507,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235,-2664.412473,-260.810113
4,2020-05-15 01:00:00,1BY6WEcLGh8j5v7,24.621525,22.165423,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,22.761668,0.000000,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235,-2664.412473,-260.810113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,2020-06-09 22:45:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,20.250877,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000,0.000000,0.000000
54908,2020-06-09 23:00:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,20.276077,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000,0.000000,0.000000
54909,2020-06-09 23:15:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,20.219128,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000,0.000000,0.000000
54910,2020-06-09 23:30:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,20.148322,0.0,0.0,-2113.138700,-206.608054,0.000000,0.000000,0.000000,0.000000


In [59]:
#groups[group].keys()

In [60]:
# add inverter difference from average per that moment
'''
* columns "MOMENT_AVERAGE" - average value per this moment of inverter production info
* columns "DIF_FROM_AVERAGE" - inverter avg values in comparison with other inverters per this moment
'''

uurimiseks= gen_data2.copy()
groups = dict(list(uurimiseks.groupby("DATE_TIME")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_DIF_AVG_PER_DAY','AC_DIF_AVG_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_DIF_AVG_PER_MOMENT'] =  groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_MOMENT'] =  groups[group]['AC_POWER'] -np.mean(groups[group]['AC_POWER']) 

    gr_nr+=1
    ''' 
    tundub valet tulemust andvat
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_MOMENT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data3=pd.DataFrame(columns =groups[group].keys())
    
    gen_data3=pd.concat([gen_data3,groups[group]], ignore_index=True)
    
gen_data3

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY,DC_DIF_AVG_PER_MOMENT,AC_DIF_AVG_PER_MOMENT
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,0.0,-1978.863369,-193.545811,-2451.465588,-240.068235,-2664.412473,-260.810113,0.0,0.0
1,2020-05-15 00:00:00,1IF53ai7Xc0U56Y,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6183645.0,...,0.0,0.0,-2138.734085,-209.096114,-2696.990699,-263.992560,-2664.412473,-260.810113,0.0,0.0
2,2020-05-15 00:00:00,3PZuoBAID5Wc2HD,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6987759.0,...,0.0,0.0,-2132.559659,-208.501252,-2653.813058,-259.767690,-2664.412473,-260.810113,0.0,0.0
3,2020-05-15 00:00:00,7JYdWkrLSPkdwr4,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,7602960.0,...,0.0,0.0,-2090.319322,-204.388230,-2610.503534,-255.561570,-2664.412473,-260.810113,0.0,0.0
4,2020-05-15 00:00:00,McdE0feGgRqW7Ca,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,7158964.0,...,0.0,0.0,-2135.935080,-208.863786,-2750.322731,-269.173270,-2664.412473,-260.810113,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,2020-06-09 23:45:00,uHbuxQJl8lW7ozc,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-2120.662268,-207.349742,0.000000,0.000000,0.000000,0.000000,0.0,0.0
54908,2020-06-09 23:45:00,wCURE6d3bPkepu2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-2120.457160,-207.322174,0.000000,0.000000,0.000000,0.000000,0.0,0.0
54909,2020-06-09 23:45:00,z9Y9gH1T5YWrNuG,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-2103.593885,-205.679726,0.000000,0.000000,0.000000,0.000000,0.0,0.0
54910,2020-06-09 23:45:00,zBIq5rxdHJRwDNY,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-2072.156565,-202.636130,0.000000,0.000000,0.000000,0.000000,0.0,0.0


#other labels adding
'''
* label "INCORRECT_total_yield" - yield per period decreases
* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off
* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not
'''

In [61]:
#Test andmestik (drop first three train days)
#gen_data3=gen_data3[6336:]

#Train andmestik - nill

In [62]:
gen_data3.to_csv(path_or_buf='Data/Train_Plant1_added_features.csv')

In [63]:
#katse=pd.read_csv("Data/Test_Plant1_added_features.csv")

In [64]:
#katse